### Binary Classifier Using Transfer Learning

- This model classifies between rotten and fresh orange by using
InceptionV3 as a pretrained model.



### Downloading **Fruits** dataset from google drive


In [2]:
!gdown 1TXwVYM3K69ocfj1t8wCS31OjoA0ajlRl

Downloading...
From: https://drive.google.com/uc?id=1TXwVYM3K69ocfj1t8wCS31OjoA0ajlRl
To: /content/fruits.zip
100% 981M/981M [00:06<00:00, 146MB/s] 


### Extracting zip file

In [15]:
import zipfile

local_zip = './fruits.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('.')
zip_ref.close()

### Setting directories

In [4]:
import os

train_dir = os.path.join('./fruits/train')
validation_dir = os.path.join('./fruits/validation')

### Removing other types of fruits in the dataset

In [30]:
import os, shutil
folder = './fruits/train'
filenames = ['fresh_apple', 'stale_apple']
folders = [train_dir, validation_dir]
for folder in folders:
  for filename in filenames:
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))

### Setting up the pretrained model (InceptionV3)

In [31]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-09-04 09:29:31--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 173.194.197.128, 64.233.191.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   262MB/s    in 0.3s    

2023-09-04 09:29:31 (262 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [32]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

# Freezing the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

### We will use the pretrained model up to (mixed7) layer as our last layer to make this model more generalized

In [33]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

### Adding new dense layers for the classifier

In [34]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

In [35]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

### Setting up the ImageDataGenerator

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalizing images
train_datagen = ImageDataGenerator(rescale=1.0/255)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_gen = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2580 images belonging to 2 classes.
Found 472 images belonging to 2 classes.


### Training the model

In [44]:
history = model.fit(
    train_gen,
    steps_per_epoch=50,
    epochs=5,
    verbose=2,
    validation_data=validation_gen,
    validation_steps=20)

Epoch 1/5
50/50 - 6s - loss: 6.2575e-04 - accuracy: 1.0000 - val_loss: 0.0096 - val_accuracy: 0.9975 - 6s/epoch - 119ms/step
Epoch 2/5
50/50 - 6s - loss: 4.8794e-04 - accuracy: 1.0000 - val_loss: 0.0167 - val_accuracy: 0.9900 - 6s/epoch - 125ms/step
Epoch 3/5
50/50 - 5s - loss: 3.4318e-04 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 0.9975 - 5s/epoch - 105ms/step
Epoch 4/5
50/50 - 6s - loss: 1.9154e-04 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 0.9950 - 6s/epoch - 126ms/step
Epoch 5/5
50/50 - 6s - loss: 2.5418e-04 - accuracy: 1.0000 - val_loss: 0.0116 - val_accuracy: 0.9950 - 6s/epoch - 127ms/step


### You can upload any image here to test the model

In [49]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array


uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = load_img(path, target_size=(150, 150))
  y = img_to_array(img)
  y = np.expand_dims(y, axis=0)

  images = np.vstack([y])
  classes = model.predict(images, batch_size=1)
  print(classes)
  print(fn)
  if(classes.flatten() == 0):
    print("This is a rotten orange")
  else:
    print("This is a fresh orange")

Saving download.jfif to download (5).jfif
1/1 [==============================] - 0s 26ms/step
[[0.]]
download (5).jfif
This is a rotten orange
